In [12]:
from ultralytics import YOLO
import cv2
import os

# Load YOLO model for SOV detection
sov_model_path = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Models/yolo_training_SOV/yolov8_sov/weights/best.pt"
sov_model = YOLO(sov_model_path)

# Paths
input_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented"  # Original images
label_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/labels/train_augmented"  # Original labels
cropped_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/images/train"  # Folder for cropped images
cropped_label_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/labels/train"  # Folder for new labels

# Create output directories
os.makedirs(cropped_folder, exist_ok=True)
os.makedirs(cropped_label_folder, exist_ok=True)

# Process each image
for frame_file in sorted(os.listdir(input_folder)):
    if frame_file.lower().endswith(('.jpg', '.png', '.jpeg')):
        frame_path = os.path.join(input_folder, frame_file)
        label_path = os.path.join(label_folder, frame_file.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Run YOLO inference for SOV detection
        results = sov_model(frame_path)

        # Get detected objects
        boxes = results[0].boxes  # Bounding boxes
        confs = boxes.conf if boxes is not None else []  # Confidence scores

        if len(confs) > 0:
            max_conf_idx = confs.argmax().item()  # Get highest confidence box index
            best_box = boxes[max_conf_idx]  # Get best bounding box

            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, best_box.xyxy[0])  # Convert to integers

            # Load the original image
            img = results[0].orig_img
            h, w, _ = img.shape  # Get image dimensions

            # Step 1: Crop image **above** the detected SOV
            cropped_img = img[:y2, :]  # Keep everything above the detected SOV
            new_h = y2  # New height of the cropped image

            # Save cropped image
            cropped_path = os.path.join(cropped_folder, frame_file)
            cv2.imwrite(cropped_path, cropped_img)

            # Step 2: Adjust label file for the cropped image
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    lines = f.readlines()

                new_labels = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id, x_center, y_center, width, height = map(float, parts)

                        # Keep only class `0` (Ascending Aorta)
                        if int(class_id) == 0:
                            # Convert normalized coordinates to pixel values
                            abs_y_center = y_center * h
                            abs_height = height * h

                            # Keep only objects that are in the cropped region (above y1)
                            if abs_y_center - (abs_height / 2) < y2:  
                                # Adjust y_center and height relative to new cropped image
                                new_y_center = abs_y_center / new_h
                                new_height = abs_height / new_h

                                # Ensure values remain within [0,1]
                                new_labels.append(f"{int(class_id)} {x_center:.6f} {new_y_center:.6f} {width:.6f} {new_height:.6f}\n")

                # Save new label file if there are valid bounding boxes
                new_label_path = os.path.join(cropped_label_folder, frame_file.replace('.jpg', '.txt').replace('.png', '.txt'))
                if new_labels:  # Save only if there are labels
                    with open(new_label_path, "w") as f:
                        f.writelines(new_labels)

print(f"✅ Cropping & label adjustment complete! New images saved in {cropped_folder}, labels in {cropped_label_folder}.")



image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented/A A - 3mensio Screen Recording_longitudinal_view_471.png: 640x640 1 SOV, 2.9ms
Speed: 1.7ms preprocess, 2.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented/A A - 3mensio Screen Recording_longitudinal_view_471_blur.jpg: 640x640 1 SOV, 3.7ms
Speed: 1.6ms preprocess, 3.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)



image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented/A A - 3mensio Screen Recording_longitudinal_view_471_brightness_contrast.jpg: 640x640 1 SOV, 3.1ms
Speed: 1.6ms preprocess, 3.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented/A A - 3mensio Screen Recording_longitudinal_view_471_clahe.jpg: 640x640 1 SOV, 2.9ms
Speed: 1.5ms preprocess, 2.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmented/A A - 3mensio Screen Recording_longitudinal_view_471_scaling.jpg: 640x640 1 SOV, 2.9ms
Speed: 1.4ms preprocess, 2.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/train_augmente

In [13]:
import cv2
import os

# Provide the specific image and label file paths
image_path = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/images/train/A A - 3mensio Screen Recording_longitudinal_view_471_blur.jpg"  # Change this
label_path = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/labels/train/A A - 3mensio Screen Recording_longitudinal_view_471_blur.txt"  # Change this

output_path = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/visualized/example_annotated.jpg"  # Save path

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Load the image
img = cv2.imread(image_path)
h, w, _ = img.shape  # Get image dimensions

# Check if label file exists
if os.path.exists(label_path):
    with open(label_path, "r") as f:
        lines = f.readlines()

    # Draw bounding boxes on the image
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id, x_center, y_center, width, height = map(float, parts)

            # Convert normalized YOLO format to absolute pixel coordinates
            x1 = int((x_center - width / 2) * w)
            y1 = int((y_center - height / 2) * h)
            x2 = int((x_center + width / 2) * w)
            y2 = int((y_center + height / 2) * h)

            # Draw bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Class {int(class_id)}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Save the image with bounding boxes
cv2.imwrite(output_path, img)
print(f"✅ Annotated image saved at: {output_path}")

✅ Annotated image saved at: /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/visualized/example_annotated.jpg


In [14]:
from ultralytics import YOLO
import cv2
import os

# Load YOLO model for SOV detection
sov_model_path = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Models/yolo_training_SOV/yolov8_sov/weights/best.pt"
sov_model = YOLO(sov_model_path)

# Paths
input_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val"  # Original images
label_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/labels/val"  # Original labels
cropped_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/images/val"  # Folder for cropped images
cropped_label_folder = "/mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/aorta_data/labels/val"  # Folder for new labels

# Create output directories
os.makedirs(cropped_folder, exist_ok=True)
os.makedirs(cropped_label_folder, exist_ok=True)

# Process each image
for frame_file in sorted(os.listdir(input_folder)):
    if frame_file.lower().endswith(('.jpg', '.png', '.jpeg')):
        frame_path = os.path.join(input_folder, frame_file)
        label_path = os.path.join(label_folder, frame_file.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Run YOLO inference for SOV detection
        results = sov_model(frame_path)

        # Get detected objects
        boxes = results[0].boxes  # Bounding boxes
        confs = boxes.conf if boxes is not None else []  # Confidence scores

        if len(confs) > 0:
            max_conf_idx = confs.argmax().item()  # Get highest confidence box index
            best_box = boxes[max_conf_idx]  # Get best bounding box

            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, best_box.xyxy[0])  # Convert to integers

            # Load the original image
            img = results[0].orig_img
            h, w, _ = img.shape  # Get image dimensions

            # Step 1: Crop image **above** the detected SOV
            cropped_img = img[:y2, :]  # Keep everything above the detected SOV
            new_h = y2  # New height of the cropped image

            # Save cropped image
            cropped_path = os.path.join(cropped_folder, frame_file)
            cv2.imwrite(cropped_path, cropped_img)

            # Step 2: Adjust label file for the cropped image
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    lines = f.readlines()

                new_labels = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id, x_center, y_center, width, height = map(float, parts)

                        # Keep only class `0` (Ascending Aorta)
                        if int(class_id) == 0:
                            # Convert normalized coordinates to pixel values
                            abs_y_center = y_center * h
                            abs_height = height * h

                            # Keep only objects that are in the cropped region (above y1)
                            if abs_y_center - (abs_height / 2) < y2:  
                                # Adjust y_center and height relative to new cropped image
                                new_y_center = abs_y_center / new_h
                                new_height = abs_height / new_h

                                # Ensure values remain within [0,1]
                                new_labels.append(f"{int(class_id)} {x_center:.6f} {new_y_center:.6f} {width:.6f} {new_height:.6f}\n")

                # Save new label file if there are valid bounding boxes
                new_label_path = os.path.join(cropped_label_folder, frame_file.replace('.jpg', '.txt').replace('.png', '.txt'))
                if new_labels:  # Save only if there are labels
                    with open(new_label_path, "w") as f:
                        f.writelines(new_labels)

print(f"✅ Cropping & label adjustment complete! New images saved in {cropped_folder}, labels in {cropped_label_folder}.")



image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val/RIVAS SEGURA, LUIS - 3mensio Screen Recording_longitudinal_view_314.png: 640x576 1 SOV, 3.6ms
Speed: 1.7ms preprocess, 3.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 576)



image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val/RIVAS SEGURA, LUIS - 3mensio Screen Recording_longitudinal_view_377.png: 640x576 1 SOV, 3.3ms
Speed: 1.6ms preprocess, 3.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val/S B - 3mensio Screen Recording_longitudinal_view_1003.png: 640x608 1 SOV, 4.6ms
Speed: 2.7ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val/S B - 3mensio Screen Recording_longitudinal_view_845.png: 640x608 1 SOV, 3.3ms
Speed: 1.7ms preprocess, 3.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 /mnt/nvme_disk2/User_data/nb57077k/cardiovision/phase2/Dataset/Original Data/images/val/S B - 3mensio Screen Recording_longitudinal_view_941.png: 640x6